In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
#load the datasets
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

testing_data = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

In [ ]:
#analyzing / understanding datasets
print(training_data.data.shape)
print(testing_data.data.shape)

image, label = training_data[0]
print(image.shape, label)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])
torch.Size([1, 28, 28]) 5


In [ ]:
#create DataLoaders for both datasets
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)

In [ ]:
#learning how DataLoaders work
data_iter = iter(training_dataloader)

while True:
    try:
      image, label = next(data_iter)
      print(image.shape, label)
    except StopIteration:
      break


In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    #Let's mimic the LeNet architecture
    self.conv1 = nn.Conv2d(1, 32, 3) #output shape = 26x26x32
    self.pool1 = nn.MaxPool2d(2, 2) #output shape = 13x13x32
    self.conv2= nn.Conv2d(32, 64, 3) #output shape = 11x11x64
    self.pool2 = nn.MaxPool2d(2, 2) #output shape = 5x5x64
    self.fc1 = nn.Linear(5*5*64, 4608) #first fc layer
    self.fc2 = nn.Linear(4608, 10) #output layer to classify 10 digits

  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))
    x = self.pool2(F.relu(self.conv2(x)))
    x = x.view(-1, 5*5*64)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
def train(epoch):
    model.train()
    running_loss = 0.0

    for (images, labels) in training_dataloader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()

      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(training_dataloader)}")

def validate(epoch):
  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in testing_dataloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      _, predicted = torch.max(outputs.data, 1) #fetch predicted value
      total += labels.size(0) #track total number of samples
      correct += (predicted == labels).sum().item() #track correct predictions

    accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss/len(testing_dataloader)}, Accuracy: {accuracy}%")

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train(epoch)
    validate(epoch)

print("Training complete")

Epoch 1/10
Epoch 1/10, Loss: 0.9089181232951216
Validation Loss: 0.31046995083997203, Accuracy: 90.69%
Epoch 2/10
Epoch 2/10, Loss: 0.2421455085992432
Validation Loss: 0.21130713137092105, Accuracy: 93.34%
Epoch 3/10
Epoch 3/10, Loss: 0.15945094818277145
Validation Loss: 0.12179995527502838, Accuracy: 96.35%
Epoch 4/10
Epoch 4/10, Loss: 0.11895404360406021
Validation Loss: 0.08938162365726604, Accuracy: 97.38%
Epoch 5/10
Epoch 5/10, Loss: 0.09728459076983716
Validation Loss: 0.0817339878031023, Accuracy: 97.6%
Epoch 6/10
Epoch 6/10, Loss: 0.08330662681190952
Validation Loss: 0.067987946137952, Accuracy: 97.92%
Epoch 7/10
Epoch 7/10, Loss: 0.07396662203438167
Validation Loss: 0.06419518417055914, Accuracy: 97.88%
Epoch 8/10


KeyboardInterrupt: 

In [ ]:
#save the model to google drive
drive.mount('/content/drive')
model_save_path = '/content/drive/MyDrive/cnn_digit_model.pth'
torch.save(model.state_dict(), model_save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Now lets end the current session so I can try loading the model
model_load_path = '/content/drive/MyDrive/cnn_digit_model.pth'
model.load_state_dict(torch.load(model_load_path))
model.eval()

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=4608, bias=True)
  (fc2): Linear(in_features=4608, out_features=10, bias=True)
)